In [1]:
# %load test.py
import numpy as np
import pandas as pd
from pathlib import Path
from keras.models import load_model
from keras.preprocessing import sequence
import pickle
import jieba
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Activation, Dropout, Input
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

try:
    from lstm.pre_process import load_data
except ImportError:
    from pre_process import load_data

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns

# %matplotlib inline
from texttable import Texttable

sns.set(font="simhei")

max_len = 300

test_set = "sub-THUCNews"

l = ['体育', '娱乐', '家居', '彩票', '房产', '教育', '时尚', '时政', '星座', '游戏', '社会', '科技', '股票', '财经']
CLASSES = {l[i]: i for i, _ in enumerate(l)}
re_CLASSES = {i:l[i] for i, _ in enumerate(l)}


def show_confusion_matrix(evaluation):
    f, ax = plt.subplots(figsize=(12, 10))
    df = pd.DataFrame(confusion_matrix(evaluation['y_true'], evaluation['y_pred']), columns=CLASSES, index=CLASSES)
    # print(df)
    ax = sns.heatmap(df, annot=True, robust=True, fmt="d", cmap="Oranges", linewidths=0.5)
    # ax.set_ylim(0.5, 0.5)
    print(ax.get_ylim())
    ax.set_ylim(14, 0)
    ax.xaxis.tick_top()
    ax.tick_params(direction='in')
    label_y = ax.get_yticklabels()
    plt.setp(label_y, rotation=360, horizontalalignment='right', fontsize=13)
    label_x = ax.get_xticklabels()
    plt.setp(label_x, horizontalalignment='center', fontsize=13)
    f.savefig("confusion_matrix.png", bbox_inches='tight', dpi=200)
    plt.show()


def show_statistics(evaluation):
    p, r, f1, s = precision_recall_fscore_support(evaluation['y_true'], evaluation['y_pred'])
    avg_p = np.average(p, weights=s)
    avg_r = np.average(r, weights=s)
    avg_f1 = np.average(f1, weights=s)
    total_s = np.sum(s)
    df1 = pd.DataFrame({'类别': l, '准确率': p, '召回率': r, 'F-measure': f1, '数量': s})
    df2 = pd.DataFrame({'类别': ['总体'], '准确率': [avg_p], '召回率': [avg_r], 'F-measure': [avg_f1], '数量': [total_s]})
    df2.index = [14]
    df = pd.concat([df1, df2])
    tb = Texttable()
    print(df)
    tb.set_cols_align(['l', 'r', 'r', 'r', 'r'])
    # tb.set_cols_dtype(['t','i','i'])
    tb.header(df.columns.get_values())
    tb.add_rows(df.values, header=False)
    print(tb.draw())


def predict(testfile):
    label_test, _, input_test, _ = load_data(testfile)

    model = load_model('model/lstm.h5')
    tok = pickle.load(open('model/token.pickle', 'rb'))

    test_seq = tok.texts_to_sequences(input_test)
    # print(input_test)
    test_seq_mat = sequence.pad_sequences(test_seq, maxlen=max_len)
    pre_result = model.predict(test_seq_mat)
    # print(pre)
    confidence = dict()
    y_pred = []
    for i,sent in enumerate(pre_result):
        p = re_CLASSES[np.argmax(sent)]
        # label = label_test[i]
        y_pred.append(p)

        # print(label, p)

    # max_index = np.argmax(pre)
    # max_type = l[max_index]
    result = {"y_true": label_test.to_list(), "y_pred": y_pred}
    # print(result)
    return result


r = predict(test_set)
show_confusion_matrix(r)
show_statistics(r)



Using TensorFlow backend.


unzip sub-THUCNews.csv.xz to sub-THUCNews.csv
load sub-THUCNews finish.


AttributeError: 'str' object has no attribute 'decode'

In [2]:
%cd ..

E:\textclassification_nlp


E:\textclassification_nlp\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
import os
print(os.path.abspath(os.path.dirname(__file__)))

NameError: name '__file__' is not defined

In [6]:
# %load lstm/predict.py
# predict
from keras.models import load_model
from keras.preprocessing import sequence
import pickle
import jieba
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

try:
    from lstm.pre_process import load_data
except ImportError:
    from .pre_process import load_data

max_len = 1000
csv_data = "THUCNews"
test_data = "sub-THUCNews"

# label, data, tokenize, length = load_data()
# input_data, input_label, _, _ = train_test_split(tokenize, label, test_size=0.9)

text = """
北京时间12月18日，2019年东亚杯冠军产生，韩国队1-0击败日本队，以3战全胜的战绩历史上第5次获得东亚杯的冠军，成为首支东亚杯3连冠球队！虽然世界杯和亚洲杯的表现不如对手，但这一次韩国队找回场子，此外在去年亚运会以及今年世青赛，韩国国奥与韩国国青都曾击败日本队，3条战线都击败对手。
"""


def predict(text):
    stopwords = [i.strip() for i in open("stopwords.txt", encoding='u8').read()]

    token = " ".join([i for i in jieba.cut(text) if i not in stopwords])
    print(token)

    model = load_model('model/lstm.h5')
    tok = pickle.load(open('model/token.pickle', 'rb'))

    test_seq = tok.texts_to_sequences([token])
    test_seq_mat = sequence.pad_sequences(test_seq, maxlen=max_len)

    """
    le = LabelEncoder()
    le_label_test = le.fit_transform(input_label).reshape(-1,1)
    
    # 分类标签转为 one hot
    ohe = OneHotEncoder()
    le_label_test = ohe.fit_transform(le_label_test).toarray()
    
    
    score, ac = model.evaluate(test_seq_mat, le_label_test, batch_size=128)
    print(score, ac)
    """
    l = ['体育', '娱乐', '家居', '彩票', '房产', '教育', '时尚', '时政', '星座', '游戏', '社会', '科技', '股票', '财经']
    confidence = dict()

    pre = model.predict(test_seq_mat)
    for i, j in enumerate(l):
        confidence[j] = pre[0][i]

    max_index = np.argmax(pre)
    max_type = l[max_index]
    confidence = {k: v for k, v in sorted(confidence.items(), key=lambda item: item[1], reverse=True)}
    return max_type, confidence


if __name__ == "__main__":
    t = predict(text)
    print(t)


FileNotFoundError: [Errno 2] No such file or directory: 'stopwords.txt'